<a href="https://colab.research.google.com/github/joshikavita01/streamlit-demo/blob/master/Assignment3_Ques3(a).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==1.14

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/cityscapes_data.zip" -d "/content/" 

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os
import shutil
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Conv2DTranspose, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

In [ ]:
import  tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
if os.path.exists('/content/train') == False:
  os.mkdir('/content/train')
if os.path.exists('/content/val/') == False:
  os.mkdir('/content/val/')



In [ ]:
for folder in os.listdir(-'/content/cityscapes_data'):
  _path = os.path.join('/content/cityscapes_data', folder)
  for i in os.listdir(_path):
    img_src = os.path.join(_path, i)
    imgge = cv2.imread(img_src)
    h, w, c = imgge.shape
    x = imgge[0:h, 0:w//2]
    y = imgge[0:h, w//2:w]
    cv2.imwrite('/content/{}/{}'.format(folder, i),x)


In [ ]:
import re
import random
import numpy as np
import os.path
import scipy.misc
import shutil
import zipfile
import time
from glob import glob
from urllib.request import urlretrieve
from tqdm import tqdm


class DL_Progress(tqdm):
	last_block = 0

	def ho_ok(self, block_num=1, block_size=1, total_size=None):
		
		self.total = total_size
		self.update((block_num - self.last_block) * block_size) 
		self.last_block = block_num


def pretrained_vgg(data_dir):
	
	_filename = 'vgg.zip'
	_path = os.path.join(data_dir, 'vgg')
	vgg_files = [
		os.path.join(_path, 'variables/variables.data-00000-of-00001'),
		os.path.join(_path, 'variables/variables.index'),
		os.path.join(_path, 'saved_model.pb')]

	missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
	if missing_vgg_files:
		
		if os.path.exists(_path):
			shutil.rmtree(_path)
		os.makedirs(_path)

	
		with DL_Progress(unit='B', unit_scale=True, miniters=1) as pbar:
			urlretrieve(
				'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
				os.path.join(_path, _filename),
				pbar.hook)
		zip_ref = zipfile.ZipFile(os.path.join(_path, _filename), 'r')
		zip_ref.extractall(data_dir)
		zip_ref.close()
		os.remove(os.path.join(_path, _filename))


def batch_(data_folder, image_shape):
	
	def batches_fn(batch_size):
		
		
		_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
		label_paths = {
			re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
			for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
		bg_color = np.array([255, 0, 0])

		
		random.shuffle(_paths)
		
		for batch_i in range(0, len(_paths), batch_size):
			images = []
			_images = []
			for image_file in _paths[batch_i:batch_i+batch_size]:
				_image_file = label_paths[os.path.basename(image_file)]
				image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
				gt_image = scipy.misc.imresize(scipy.misc.imread(_image_file), image_shape)
				gt_bgg = np.all(gt_image == bg_color, axis=2)
				gt_bgg = gt_bgg.reshape(*gt_bgg.shape, 1)
				gt_image = np.concatenate((gt_bgg, np.invert(gt_bgg)), axis=2)

				images.append(image)
				_images.append(gt_image)

			yield np.array(images), np.array(_images)
	return batches_fn


def testt_output(sess, logits, keep_prob, image_pl, data_folder, image_shape):
	
	for _file in glob(os.path.join(data_folder, 'image_2', '*.png')):
		i = scipy.misc.imresize(scipy.misc.imread(_file), image_shape)

		
		im_softmax = sess.run(
			[tf.nn.softmax(logits)],
			{keep_prob: 1.0, image_pl: [i]})
		
		_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
	
		seg = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
		
		m = np.dot(seg, np.array([[0, 255, 0, 127]]))
		m = scipy.misc.toimage(m, mode="RGBA")
		street_im = scipy.misc.toimage(i)
		street_im.paste(m, box=None, mask=m)

		yield os.path.basename(_file), np.array(street_im)


def _samples(_dir, data_, s, _shape, logits, _prob, input_):
	output_dir = os.path.join(_dir, str(time.time()))
	if os.path.exists(_dir):
		shutil.rmtree(_dir)
	os.makedirs(_dir)

	print(' Save test images: {}'.format(_dir))
  _outputs = testt_output(
		sess, logits, keep_prob, _image, os.path.join(data_, 'data_road/testing'), _shape)
	for name, image in _outputs:
		scipy.misc.imsave(os.path.join(_dir, name), image)

In [ ]:
EPOCHS = 60
BATCH_SIZE = 25
NUM_CLASSES=29
IS_TRAIN = True
KEEP_PROB = 0.5
LEARNING_RATE = 0.0005
L2_REG = 1e-2
STDEV = 1e-3

DATA_DIR = '/content/train'
VIDEO_DIR = 'data\leftImg8bit_demoVideo\leftImg8bit\demoVideo\stuttgart_02'
RUNS_DIR = 'runs'

DEFAULT_SAVE_DIR = ''
PRETRAINED_MODEL_DIR = ''

In [ ]:
import cv2
from collections import namedtuple
import random
import numpy as np
import os.path
import scipy.misc
import os
from glob import glob
from sklearn.utils import shuffle
import random
import time

def label_info():
    Label = namedtuple('Label', [

        'name',  
        'id',  

        'trainId',  

        'category',  

        'categoryId', 

        'hasInstances', 

        'ignoreInEval',  

        'color', 
    ])

    l= [
        Label('unlabeled', 0, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('ego vehicle', 1, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('rectification border', 2, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('out of roi', 3, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('static', 4, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('dynamic', 5, 255, 'void', 0, False, True, (111, 74, 0)),
        Label('ground', 6, 255, 'void', 0, False, True, (81, 0, 81)),
        Label('road', 7, 0, 'flat', 1, False, False, (128, 64, 128)),
        Label('sidewalk', 8, 1, 'flat', 1, False, False, (244, 35, 232)),
        Label('parking', 9, 255, 'flat', 1, False, True, (250, 170, 160)),
        Label('rail track', 10, 255, 'flat', 1, False, True, (230, 150, 140)),
        Label('building', 11, 2, 'construction', 2, False, False, (70, 70, 70)),
        Label('wall', 12, 3, 'construction', 2, False, False, (102, 102, 156)),
        Label('fence', 13, 4, 'construction', 2, False, False, (190, 153, 153)),
        Label('guard rail', 14, 255, 'construction', 2, False, True, (180, 165, 180)),
        Label('bridge', 15, 255, 'construction', 2, False, True, (150, 100, 100)),
        Label('tunnel', 16, 255, 'construction', 2, False, True, (150, 120, 90)),
        Label('pole', 17, 5, 'object', 3, False, False, (153, 153, 153)),
        Label('polegroup', 18, 255, 'object', 3, False, True, (153, 153, 153)),
        Label('traffic light', 19, 6, 'object', 3, False, False, (250, 170, 30)),
        Label('traffic sign', 20, 7, 'object', 3, False, False, (220, 220, 0)),
        Label('vegetation', 21, 8, 'nature', 4, False, False, (107, 142, 35)),
        Label('terrain', 22, 9, 'nature', 4, False, False, (152, 251, 152)),
        Label('sky', 23, 10, 'sky', 5, False, False, (70, 130, 180)),
        Label('person', 24, 11, 'human', 6, True, False, (220, 20, 60)),
        Label('rider', 25, 12, 'human', 6, True, False, (255, 0, 0)),
        Label('car', 26, 13, 'vehicle', 7, True, False, (0, 0, 142)),
        Label('truck', 27, 14, 'vehicle', 7, True, False, (0, 0, 70)),
        Label('bus', 28, 15, 'vehicle', 7, True, False, (0, 60, 100)),
        Label('caravan', 29, 255, 'vehicle', 7, True, True, (0, 0, 90)),
        Label('trailer', 30, 255, 'vehicle', 7, True, True, (0, 0, 110)),
        Label('train', 31, 16, 'vehicle', 7, True, False, (0, 80, 100)),
        Label('motorcycle', 32, 17, 'vehicle', 7, True, False, (0, 0, 230)),
        Label('bicycle', 33, 18, 'vehicle', 7, True, False, (119, 11, 32)),
        Label('license plate', -1, -1, 'vehicle', 7, False, True, (0, 0, 142)),
    ]

    s = set()
    _list = list(map(lambda x : x[7], l))
    _values = [x for x in _list if not (x in s or s.add(x))]

    return _values

def get_data(dp):
    tp = dp + '/leftImg8bit/train/'
    typ = dp+ '/sky-data/train/'

    vp = dp + '/leftImg8bit/val/'
    vyp = dp + '/sky-data/val/'

    tb = glob(os.path.join(tp, '*/*.png'))

    tb = glob(os.path.join(typ, '*/*color.png'))
    vp = glob(os.path.join(vp, '*/*.png'))

    vyp = glob(os.path.join(vyp, '*/*color.png'))

    X_tr = []
    y_tr = []
    X_v = []
    y_v = []

    
    for s in tb:
        img_path = s
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        X_tr.append(x)

    
    for s in typ:
        img_path = s
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        y_tr.append(x)

    
    for s in vp:
        img_path = s
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        X_v.append(x)

    
    for s in vyp:
        img_path = s
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        y_v.append(x)

    return X_tr, y_tr, X_v, y_v


  

In [ ]:
import helper
import warnings
from distutils.version import LooseVersion
def load_vgg(sess, vgg_path):
   
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    graph = tf.get_default_graph()
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return input, keep_prob, layer3, layer4, layer7

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    l7_conv = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       kernel_regularizer=tf.keras.regularizers.L2(0.01), name='conv_1_1_1',activation = tf.nn.relu)


    conv1 = tf.layers.conv2d_transpose(l7_conv, num_classes, 4, 2,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        kernel_regularizer=tf.keras.regularizers.L2(0.01), name='conv_1_1_2',activation = tf.nn.relu)

    l4_conv = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       kernel_regularizer=tf.keras.regularizers.L2(0.01), name='conv_1_1_3',activation = tf.nn.relu)

    skip_1 = tf.add(conv1, l4_conv, name='conv_1_1_4')
    conv2 = tf.layers.conv2d_transpose(skip_1, num_classes, 4, 2,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        kernel_regularizer=tf.keras.regularizers.L2(0.01),  name='conv_1_1_5',activation = tf.nn.relu)
    l3_conv = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       kernel_regularizer=tf.keras.regularizers.L2(0.01), name='conv_1_1_6',activation = tf.nn.relu)
    skip_3 = tf.add(conv2, l3_conv,  name='conv_1_1_7')

    output = tf.layers.conv2d_transpose(skip_3, num_classes, 16, 8,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        kernel_regularizer=tf.keras.regularizers.L2(0.01),  name='conv_1_1_8',activation = tf.nn.relu)

    return output

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    

    logits = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    labels = tf.reshape(correct_label, (-1, num_classes), name='labels')
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

    train_op = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss


import time


def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate, X_train, y_train, label_values, X_val, y_val):
    
    for epoch in range(epochs):
        print('epoch : ', epoch)
        for image, targets in gbf(X_train, y_train, label_values, batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image: image, correct_label: targets, keep_prob: KEEP_PROB,
                                          learning_rate: LEARNING_RATE})

            print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Loss: {:.3f}".format(loss))

        mean_loss = []
        for image, targets in get_batches_fn(X_val, y_val, label_values, batch_size, is_train=False):
            loss = sess.run([cross_entropy_loss],
                            feed_dict={input_image: image, correct_label: targets, keep_prob: 1})
            mean_loss.append(loss)

        mean_loss_ = np.mean(np.array(mean_loss))
        print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Validation Loss: {:.3f}".format(mean_loss_))
        saver = tf.train.Saver()
        print('saving model')
        saver.save(sess, './model')





def run():
    num_classes = NUM_CLASSES
    image_shape = (512, 256)  
    data_dir = DATA_DIR
    video_dir = VIDEO_DIR
    runs_dir = RUNS_DIR

    epochs = EPOCHS
    batch_size = BATCH_SIZE
    is_train = IS_TRAIN
   
    maybe_download_pretrained_vgg(data_dir)
    label_values = get_label_info()

    tf.reset_default_graph()
    with tf.Session() as sess:
        
        vgg_path = os.path.join(data_dir, 'vgg')
        
        gbf = gen_batch_function

        input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        print(input)
        output = layers(layer3, layer4, layer7, num_classes)
        correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes))
        learning_rate = tf.placeholder(dtype=tf.float32)
        logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)
        logits = tf.nn.softmax(logits, name='softmax')
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver() 

        if not is_train:
            saver.restore(sess, tf.train.latest_checkpoint('.'))
            save_inference_samples(runs_dir, video_dir, sess, image_shape, logits, keep_prob, input, label_values)
        else:
            X_train, y_train, X_val, y_val = get_data(data_dir)
            train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input, correct_label,
                         keep_prob, learning_rate, X_train, y_train, label_values, X_val, y_val)
            save_inference_samples(runs_dir, video_dir, sess, image_shape, logits, keep_prob, input,
                                                     label_values)
if __name__ == '__main__':
    run()

TensorFlow Version: 2.8.0
Default GPU Device: /device:GPU:0
run
INFO:tensorflow:Restoring parameters from /content/train/vgg/variables/variables
Tensor("image_input:0", shape=(?, ?, ?, 3), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: UserWarning: `tf.layers.conv2d_transpose` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2DTranspose` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:1736: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: `tf.layers.conv2d` is deprecat

Loading X_Train..
Loading Y_Train..
Loading X_Validation..
Loading Y_Validation..
epoch :  0
Epoch: 1 / 60  Validation Loss: nan
saving model


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


epoch :  1
Epoch: 2 / 60  Validation Loss: nan
saving model
epoch :  2
Epoch: 3 / 60  Validation Loss: nan
saving model
epoch :  3
Epoch: 4 / 60  Validation Loss: nan
saving model
epoch :  4
Epoch: 5 / 60  Validation Loss: nan
saving model
epoch :  5
Epoch: 6 / 60  Validation Loss: nan
saving model
epoch :  6
Epoch: 7 / 60  Validation Loss: nan
saving model
epoch :  7
Epoch: 8 / 60  Validation Loss: nan
saving model
epoch :  8
Epoch: 9 / 60  Validation Loss: nan
saving model
epoch :  9
Epoch: 10 / 60  Validation Loss: nan
saving model
epoch :  10
Epoch: 11 / 60  Validation Loss: nan
saving model
epoch :  11
Epoch: 12 / 60  Validation Loss: nan
saving model
epoch :  12
Epoch: 13 / 60  Validation Loss: nan
saving model
epoch :  13
Epoch: 14 / 60  Validation Loss: nan
saving model
epoch :  14
Epoch: 15 / 60  Validation Loss: nan
saving model
epoch :  15
Epoch: 16 / 60  Validation Loss: nan
saving model
epoch :  16
Epoch: 17 / 60  Validation Loss: nan
saving model
epoch :  17
Epoch: 18 / 6

In [ ]:
os.mkdir('/content/saved_op')